# Text Classifier

Chapter 5 of Programming PyTorch for Deep Learning, but using samples from the [TREC 2005 Spam Corpus](https://trec.nist.gov/data/spam.html) instead of tweets.

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data 
import torchtext

In [2]:
def my_tokenize(s):
    return s.split()

In [3]:
TEXT = data.Field(lower=True, tokenize=my_tokenize)
LABEL = data.Field(lower=True)
samples = data.TabularDataset(path='./data/ham-spam-samples.tsv',
                              format='tsv', 
                              fields=[("label",LABEL), ("statement",TEXT)],
                              skip_header=False)

In [4]:
(training, testing, validating) = samples.split(split_ratio=[0.6,0.2,0.2])
(len(training),len(testing),len(validating))

(181, 60, 60)

In [5]:
vocab_size = 60
TEXT.build_vocab(training, max_size = vocab_size)
LABEL.build_vocab(training)
TEXT.vocab.freqs.most_common(10)

[('the', 289),
 ('to', 224),
 ('for', 140),
 ('and', 136),
 ('a', 114),
 ('of', 109),
 ('you', 106),
 ('in', 98),
 ('is', 95),
 ('your', 81)]

In [6]:
len(TEXT.vocab)

62

In [7]:
vars(training.examples[1])

{'label': ['ham'],
 'statement': ['per',
  'our',
  'discussion',
  'at',
  'the',
  'controls',
  'meeting',
  '-',
  'i',
  'thought',
  'you',
  'might',
  'be',
  'interested',
  'in',
  'ken',
  "cessac's",
  'recent',
  'findings',
  'on',
  'how',
  'the',
  'field',
  'could',
  'be',
  'more',
  'proactive',
  'at',
  'finding',
  'and',
  'correcting',
  'meter',
  'errors.',
  '-----original',
  'message-----',
  'from:',
  'cessac,',
  'kenneth',
  'sent:',
  'tuesday,',
  'july',
  '03,',
  '2001',
  '6:15',
  'pm',
  'to:',
  'datta-barua,',
  'lohit;',
  'alters,',
  'dennis;',
  'anderson,',
  'gary',
  'e.;',
  'armstrong,',
  'julie;',
  'asante,',
  'ben;',
  'bellard,',
  'dannis;',
  'blair,',
  'lynn;',
  'br']}

In [8]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [9]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
(training, validating, testing), 
batch_size = 32,
device = device,
sort_key = lambda x: len(x.statement),
sort_within_batch = False)

# Defining the model

Start with a simple [Long short-term memory (LSTM)](https://en.wikipedia.org/wiki/Long_short-term_memory) model

In [10]:
class BasicLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(BasicLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1)
        self.predictor = nn.Linear(hidden_size, 1)

    def forward(self, seq):
        output, (hidden, _) = self.encoder(self.embedding(seq))
        preds = self.predictor(hidden.squeeze(0))
        return preds.double()

In [11]:
def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    m = nn.Sigmoid()
    for epoch in range(1, epochs + 1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.statement)
            loss = criterion(m(predict), batch.label.double())
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.statement.size(0)
        training_loss /= len(train_iterator)
 
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.statement)
            loss = criterion(m(predict), batch.label.double())
            valid_loss += loss.data.item() * batch.statement.size(0)
 
        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

In [12]:
model = BasicLSTM(100, 300, 62)
model.to(device)

BasicLSTM(
  (embedding): Embedding(62, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=1, bias=True)
)

In [13]:
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.BCELoss()

In [14]:
train(5, model, optimizer, criterion, train_iterator, valid_iterator)

/home/denis/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1, 32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/home/denis/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1, 21])) that is different to the input size (torch.Size([21, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/home/denis/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1, 28])) that is different to the input size (torch.Size([28, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduct

Epoch: 1, Training Loss: -483.59, Validation Loss: -766.79
Epoch: 2, Training Loss: -1178.51, Validation Loss: -1314.51
Epoch: 3, Training Loss: -1836.56, Validation Loss: -1869.30
Epoch: 4, Training Loss: -2538.61, Validation Loss: -2424.09
Epoch: 5, Training Loss: -3062.35, Validation Loss: -2867.16


# Making Predictions

In [15]:
def classify_text(text):
    categories = {0: 'ham', 1: 'spam'}
    processed = TEXT.process([TEXT.preprocess(text)])
    processed = processed.to(device)
    return categories[model(processed).argmax().item()]

In [16]:
classify_text(testing.examples[0].statement)

'ham'

In [17]:
testing.examples[0].label[0]

'ham'

In [18]:
correct = 0
examined = 0
for test_example in testing.examples:
    actual = test_example.label[0]
    predicted = classify_text(test_example.statement)
    examined += 1
    if actual == predicted:
        correct += 1
        print('Correct   --> {}/{} right overall'.format(correct, examined))
    else:
        print('Incorrect --> {}/{} right overall'.format(correct, examined))

Correct   --> 1/1 right overall
Correct   --> 2/2 right overall
Correct   --> 3/3 right overall
Incorrect --> 3/4 right overall
Correct   --> 4/5 right overall
Correct   --> 5/6 right overall
Correct   --> 6/7 right overall
Incorrect --> 6/8 right overall
Correct   --> 7/9 right overall
Correct   --> 8/10 right overall
Correct   --> 9/11 right overall
Incorrect --> 9/12 right overall
Incorrect --> 9/13 right overall
Incorrect --> 9/14 right overall
Correct   --> 10/15 right overall
Correct   --> 11/16 right overall
Incorrect --> 11/17 right overall
Incorrect --> 11/18 right overall
Correct   --> 12/19 right overall
Correct   --> 13/20 right overall
Correct   --> 14/21 right overall
Incorrect --> 14/22 right overall
Correct   --> 15/23 right overall
Incorrect --> 15/24 right overall
Correct   --> 16/25 right overall
Correct   --> 17/26 right overall
Correct   --> 18/27 right overall
Incorrect --> 18/28 right overall
Correct   --> 19/29 right overall
Correct   --> 20/30 right overall
Cor